In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# class time_simulation

In [6]:
class time_simulation:
    def __init__(self, 
                 tree,
                 rate = 0.001,
                 regime = "univariate",
                 num_ou = 700,
                 num_bm = 300,
                 params = "random",
                 max_a = 10,
                 max_s = 10,
                 max_th = 100,
                 ou_var_threshold = 1):
        '''
        Arguments 
        tree(DataFrame) : contains nodes, species, ancestors, and time column
        rate(float) : size of time grid(=dt)
        regime(list or "univariate") : optimum regime, length should be same as the number of nodes
        num_ou(int) : number of nonalpha parameter sets
        num_bm(int) : number of zero-alpha parameter sets
        params(DataFrame or "random") : set of ouch parameters, containing alpha, sigma, and theta 
        max_a(float) : maximum alpha value for random parameter generation
        max_s(float) : maximum sigma value for random parameter generation
        max_th(float) : maximum theta value for random parameter generation
        ou_var_threshold(float) : maximum ou variance(sigma^2 / (2 alpha)) for random parameter generation
        '''

        self.tree = tree

        # delta t
        self.rate = rate
        
        # number of genes under selection(ou) or drift(bm)
        self.num_ou = num_ou
        self.num_bm = num_bm

        # regime
        if regime == "univariate":
            self.regime = [0 for i in range(tree.shape[0])]
        else:
            self.regime = self.set_regime()
        self.regime = np.array(self.regime)
        self.nreg = len(np.unique(self.regime))

        # set boundary value for random parameters
        self.max_a = max_a
        self.max_s = max_s
        self.max_th = max_th
        self.ou_var_threshold = ou_var_threshold

        # generate parameters
        if params == "random" : 
            self.params = self.rand_param()
        else:
            self.params = params


    def set_regime(self):
        '''
        generate regime list with unique numerical values
        '''
        reg = np.unique(self.regime)
        numerical_regime = [reg.index(item) for item in self.regime]

        return numerical_regime
    

    def rand_param(self):
        ## generate random parameters
        params = {}
        i = 1

        while len(params) < self.num_ou:
            alpha = np.random.uniform(0, self.max_a)
            theta = np.random.uniform(0, self.max_th, self.nreg)
            sigma = np.random.uniform(0, self.max_s)

            if sigma**2 <= alpha * 2 * self.ou_var_threshold:
                ## keep the ou variance lower than ou_var_threshold
                params[i] = [alpha, theta, sigma]
                i += 1

        for j in range(self.num_bm):
            theta = np.random.uniform(0, self.max_th, self.nreg)
            sigma = np.random.uniform(0, self.max_s)    

            params[i+j] = [0,theta, sigma]

        params = pd.DataFrame(params)
        params = params.T
        params.columns = ["alpha", "theta", "sigma"]

        return params


    def sim(self, param):
        theta = param["theta"]
        sigma = param["sigma"]
        alpha = param["alpha"]

        X = {1:theta[0]}
        for i in range(1,tree.shape[0]):
            ti = self.tree["time"][self.tree["ancestor"][i]-1]
            tf = self.tree["time"][i] 
            x = X[self.tree["ancestor"][i]]

            for j,t in enumerate(np.arange(ti, tf, self.rate)):
                x += alpha * (theta[self.regime[i]] - x) + sigma * np.random.normal(0, self.rate, 1)[0]

            X[i + 1]= x

        return X
    

    def run(self):
        data = {}
        nterm = self.tree[self.tree["species"].notna()].shape[0]

        for i in self.params.index.tolist():
            if i % 500 == 0:
                print(i)
                #print(X)
            X = self.sim(self.params.loc[i,:])
            data[i] = X

        data = pd.DataFrame(data)
        data = data.loc[nterm:,:].T
        data.columns = self.tree["species"][nterm-1:]

        return self.params, data

# input tree data

In [4]:
tree = pd.read_csv("/Users/yunseong-eun/Desktop/processed_chen/mammals.tree.txt", delimiter="\t")
tree

,node,species,ancestor,time
0,1,NaN,NaN,0.0000
1,2,NaN,1.0,0.2446
2,3,NaN,2.0,0.2671
3,4,NaN,3.0,0.3025
4,5,NaN,4.0,0.3841
5,6,NaN,3.0,0.2883
6,7,NaN,6.0,0.3013
7,8,NaN,7.0,0.5757
8,9,NaN,8.0,0.6324
9,10,NaN,9.0,0.6489


# run

In [7]:
sim = time_simulation(tree)
p, data = sim.run()
data 
# too high variance across sample of one gene (especially in non-zero alpha..)
# This might result from the zigzag-like behavior of trajectory

/var/folders/bw/bxv4t4_53sgdscvt91xk5tmc0000gn/T/ipykernel_26804/639160248.py:107: RuntimeWarning: overflow encountered in scalar multiply
  x += alpha * (theta[self.regime[i]] - x) + sigma * np.random.normal(0, self.rate, 1)[0]
/var/folders/bw/bxv4t4_53sgdscvt91xk5tmc0000gn/T/ipykernel_26804/639160248.py:107: RuntimeWarning: invalid value encountered in scalar add
  x += alpha * (theta[self.regime[i]] - x) + sigma * np.random.normal(0, self.rate, 1)[0]


500
1000


species,opossum,armadillo,cow,dog,ferret,rabbit,rat,musCaroli,musSpretus,musMusculus,marmoset,rhesus,orangutan,gorilla,chimp,bonobo,human
1,-7.196843e+233,-9.121784e+278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.857691e+302,-2.590696e+295,-4.690225e+292,-4.690225e+292,2.273785e+293,2.273785e+293,2.273785e+293
2,1.208360e+155,-1.784579e+185,1.692926e+243,1.027615e+214,-3.248035e+224,-4.384025e+233,-1.075946e+303,5.613418e+300,5.613418e+300,5.613418e+300,-5.928773e+200,-1.613759e+196,-2.408730e+194,-2.408730e+194,6.891294e+194,6.891294e+194,6.891294e+194
3,2.047588e+234,6.419036e+278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.082893e+302,1.596177e+295,2.954467e+292,2.954467e+292,-1.424392e+293,-1.424392e+293,-1.424392e+293
4,4.284593e+172,-1.948889e+205,2.184773e+269,1.154603e+237,-4.591665e+248,-5.604339e+258,NaN,NaN,NaN,NaN,-2.734823e+222,-2.475439e+217,-2.378709e+215,-2.378709e+215,7.597471e+215,7.597471e+215,7.597471e+215
5,1.698094e+294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,8.108697e+01,8.111426e+01,8.111475e+01,8.108897e+01,8.108073e+01,8.111569e+01,8.110203e+01,8.111010e+01,8.111844e+01,8.111396e+01,8.111261e+01,8.111103e+01,8.109973e+01,8.109887e+01,8.109755e+01,8.109960e+01,8.109740e+01
997,2.275776e+01,2.318207e+01,2.294385e+01,2.293898e+01,2.284294e+01,2.287482e+01,2.316427e+01,2.323894e+01,2.321443e+01,2.323244e+01,2.286813e+01,2.293651e+01,2.299528e+01,2.307594e+01,2.302147e+01,2.304410e+01,2.307142e+01
998,1.643748e+01,1.633203e+01,1.600875e+01,1.612642e+01,1.614435e+01,1.606586e+01,1.625121e+01,1.621267e+01,1.624122e+01,1.631357e+01,1.605190e+01,1.609946e+01,1.603597e+01,1.603289e+01,1.607101e+01,1.604926e+01,1.606928e+01
999,6.768075e+01,6.769257e+01,6.775222e+01,6.772522e+01,6.774307e+01,6.776226e+01,6.777014e+01,6.775957e+01,6.774235e+01,6.773523e+01,6.771766e+01,6.772357e+01,6.772360e+01,6.773866e+01,6.772291e+01,6.771869e+01,6.772124e+01
